В этом проекте мы проанализируем зависимость цен на недвижимость от множества факторов, а нашей задачей будет научиться предсказывать стоимость по имеющимся данным. Присмотримся к рынку недвижимости и возьмем на вооружение метод OLS.

Наша цель будет состоять в том, чтобы построить линейную модель. Это значит, что для набора признаков X мы хотим подобрать такие коэффициенты A, чтобы sum(A[i]*X[i]) было как можно ближе к реальной цене.

Задачи в проекте будут соответствующими:

1. вначале мы поразмыслим об эффективности вычислений;
2. затем подумаем, как применить элементы математической статистики для решении этой задачи;
3. напоследок закрепим одну из базовых операций — получение обратной матрицы.

# Задание 1

Задание-разминка. При работе с большими данными необходимо учитывать оптимальность выполнения расчётов.  

Для выполнения этого задания определите, в каком порядке эффективнее умножить три матрицы. Например, у нас есть матрицы A=[[1,2]], B=[[2], [1]], C=[[5]]. Эффективнее сначала умножить AxB (два умножения), а потом умножить результат на C (одно умножение). В этом случае мы выполним три умножения.

Если же мы будем умножать сначала B на C (два умножения), а потом результат умножим слева на A (два умножения), то в сумме будет четыре умножения.

Входные данные: матрицы A,B,C в виде numpy-массивов. Гарантируется, что их можно перемножить в порядке AxBxC.

Результат: напишите функцию multiplication_order(A, B, C), которая вернёт строку "(AxB)xC", если количество умножений элементов матриц при умножении  меньше либо равно количеству умножений, если выполнять их в порядке . В противном случае верните строку "Ax(BxC)".

Пример входных данных: A=[[1,2]], B=[[2], [1]], C=[[5]]  
Пример результата: “(AxB)xC”.


In [1]:
#ваш код
import numpy as np
import pandas as pd

def multiplication_order(A, B, C):
    # Получаем размеры матриц
    # A_nxm  B_mxk  C_k_l
    n, m = A.shape
    m, k = B.shape
    k, l = C.shape

    # Считаем количество умножений
    # AxB - это матрица размера n x k,
    # каждый элемент которой получается суммой из m произведений
    ab_mul_cnt = m * (n * k)
    # (AxB)xC это матрица размера n x l,
    # каждый элемент которой получается суммой из k произведений
    ab_c_mul_cnt = k * (n * l)
    # BxC- это матрица размера m x l,
    # каждый элемент которой получается суммой из k произведений
    bc_mul_cnt = k * (m * l)
    # Ax(BxC) - это матрица размера n x l,
    # каждый элемент которой получается суммой из m произведений
    a_bc_mul_cnt = m * (n * l)

    # находим вариант с наименьшим количеством
    # умножений элементов матриц при умножении
    if ab_mul_cnt + ab_c_mul_cnt <= bc_mul_cnt + a_bc_mul_cnt:
        return "(AxB)xC"
    else:
        return "Ax(BxC)"

# Входные данные
A = np.array([[1,2]])
B = np.array([[2],[1]])
C = np.array([[5]])

multiplication_order(A, B, C)

'(AxB)xC'

# Задание 2


В этой задаче вам потребуется найти признак, наиболее сильно коррелирующий с результатом, и найти самый «слабый» признак.

Входные данные: матрица признаков X в виде numpy-массива и вектор цен Y. Количество строк в матрице X совпадает с количеством элементов в векторе Y. Каждая строка матрицы описывает признаки одной квартиры, а соответствующий элемент в Y равен цене квартиры.

Результат: напишите функцию best_worst(X, Y), которая вернёт два числа: max_corr_idx (номер признака, наиболее коррелирующего с ценой) и min_corr_idx (номер признака, наименее коррелирующего с ценой). Учитывайте, что корреляция имеет знак, а сила корреляции зависит от абсолютного значения — нужно вернуть наибольший и наименьший признаки по абсолютному значению.

Подсказка: можно использовать функцию [numpy.corrcoef](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html) для быстрого вычисления коэффициентов корреляции.

Пример входных данных: см. таблицу из предисловия к проекту.

Пример результата: функция должна вернуть кортеж (3, 2).   
3 — номер (начиная с 0) столбца Центр?, цена больше всего зависит от того, находится ли квартира в центре. 2 — номер столбца Этаж. В этом примере от номера этажа цена практически не зависит.

In [2]:
#ваш код
#  Исходные данные
# Проверим на тестовых данных
data = np.array([
    [3, 51, 3, 0, 1, 0, 2200],
    [1, 30, 1, 0, 1, 0, 1600],
    [2, 45, 2, 0, 1, 0, 1900],
    [3, 55, 1, 0, 1, 0, 2000],
    [1, 45, 3, 1, 0, 0, 4500],
    [3, 100, 3, 1, 0, 0, 7000],
    [2, 71, 2, 1, 0, 0, 5000],
    [1, 31, 2, 0, 0, 1, 1700],
    [3, 53, 5, 0, 0, 1, 2100],
    [1, 33, 3, 0, 0, 1, 1500],
    [2, 43, 5, 0, 0, 1, 2000],
])
df = pd.DataFrame(data=data, columns=['Комнаты', 'Площадь', 'Этаж', 'Центр?', 'Спальный1?', 'Спальный 2?', 'Цена'])
# Столбцы Центр?, Спальный1 и Спальный2 говорят о районе, в котором расположена квартира.
# Значение 1, если квартира находится в данном районе, и 0 , если в другом.

X = df[['Комнаты', 'Площадь', 'Этаж', 'Центр?', 'Спальный1?', 'Спальный 2?']]
y = df[['Цена']]

def best_worst(X, Y):
    # Вычисляем коэффициенты корреляции между признаками X и ценами Y
    # Берем только последнюю строку (это корреляция с целевым вектором Y)
    # и опускаем последний столбец полученной матрицы, так как он будет равен 1 - это кореляция целевого признака с самим собой
    corr = np.corrcoef(X.T, Y.T)[-1][:-1]

    # Находим индексы максимального и минимального значений корреляций по абсолютному значению
    # номер признака, наиболее коррелирующего с ценой
    max_corr_idx = np.argmax(np.abs(corr))
    # номер признака, наименее коррелирующего с ценой
    min_corr_idx = np.argmin(np.abs(corr))

    return max_corr_idx, min_corr_idx

best_worst(X,y)

(3, 2)

#Задание 3


В предыдущей задаче мы исследовали связь признаков с результатом: стоимостью недвижимости. А можно ли что-то сказать о связях между самими признаками? Для этого в данной задаче вам потребуется найти ранг матрицы корреляции.

Применительно к нашей задаче предсказания цен на недвижимость поиск зависимых между собой признаков может быть полезен для оптимизации модели.

Напомним, что если ранг матрицы корреляции меньше её размерности, то какие-то из признаков, которые мы рассматриваем, могут быть линейно зависимы от других, а значит могут быть проигнорированы. Если же все рассматриваемые признаки обладают отдельной значимостью, то ранг матрицы корреляции будет равен её размерности. Каждая строка матрицы — это набор признаков данного объекта.

Входные данные: матрица признаков X в виде numpy-массива.

Результат: напишите функцию corr_rank(X), возвращающую одно число — ранг корреляционной матрицы.

Подсказка: можно использовать функцию np.linalg.matrix_rank.

Пример входных данных: см. таблицу из предисловия к проекту.

Пример результата: 5

In [3]:
#ваш код
def corr_rank(X):
    #строим матрицу корреляций для набора признаков X из прошлой подзадачи
    corr = np.corrcoef(X)
    # ранг матрицы корреляций
    return np.linalg.matrix_rank(corr)

corr_rank(X)

5

# Задание 4

В этой задаче по входной квадратной матрице  нужно определить, возможно ли найти обратную матрицу, и, если возможно, вернуть её.  

Входные данные: матрица A в виде numpy-массива.

Результат: напишите функцию inverse_matrix(A), которая вернёт None, если матрица необратима (то есть её определитель по абсолютному значению меньше 0.001), либо вернёт обратную матрицу в виде numpy-массива. Подсказка: используйте [np.linalg.inv](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html).

Пример входных данных: A = np.array([[1, 2], [2, 1]])

Пример результата: array([[-0.33333333, 0.66666667],[ 0.66666667, -0.33333333]])

In [4]:
#ваш код
def inverse_matrix(A):
    if np.abs(np.linalg.det(A)) < 0.001:
        return None
    return np.linalg.inv(A)

A = np.array([[1, 2], [2, 1]])

inverse_matrix(A)

array([[-0.33333333,  0.66666667],
       [ 0.66666667, -0.33333333]])

# Задание 5

Начнём процесс построения модели, которая выберет оптимальные варианты для инвестиций в недвижимость: модель сможет предсказывать цены на квартиры по набору признаков. Для этого мы воспользуемся знакомым нам методом OLS.  

Из теории вам уже известна формула $a=(X^TX)^{-1}X^Ty$, осталось её только запрограммировать.

Пояснение:  
Матрица Мура-Пенроуза $Q=(X^TX)^{-1}X^T$ — это обобщение обратной матрицы для случая матриц, не являющихся квадратными. Действительно, если матрица не квадратная, то для неё невозможно найти обратную в привычном смысле этого слова. Однако при выполнении некоторых условий нам подойдёт матрица $Q$. Она работает селдующим образом: домножим выражение $a=(X^TX)^{-1}X^Ty$ слева на $X$: $X_a=X(X^TX)^{-1}X^Ty$. Если бы матрица $X$ была квадратной и обратимой, то выражение перед $y$ просто пропало бы — сократились бы все множители: $X(X^TX)^{-1}X^T=E$ . Тогда получается, что $X_a=y$, то есть коэффициенты $a$ получились такими, что мы научились предсказывать вектор цен $y$. Оказывается, что для случая прямоугольной матрицы этот «трюк» тоже работает, хоть и нельзя формально производить раскрытие скобок.  


Входные данные: матрица признаков $X$ в виде numpy-массива и вектор цен $y$ в виде numpy-массива. $X$ имеет форму $(m, n)$ — $m$ строк, по строке на каждую квартиру, $n$ признаков для каждой квартиры. y имеет форму $(m)$, это вектор из $m$ элементов — цены всех m квартир.

Результат: напишите функцию fit_model(X, y), которая вернёт numpy-массив с оптимальными коэффициентами $a$, найденными [методом OLS](https://en.wikipedia.org/wiki/Ordinary_least_squares).

Пример входных данных: см. таблицу из предисловия к проекту.

Пример результата: [-574.12295766 65.33255763 141.80223878 1566.16246224 12.32450391 -315.34552489]

In [9]:
#ваш код
def fit_model(X, y):
    return (inverse_matrix(X.T@X)@X.T@y).to_numpy()

fit_model(X, y)

array([[-574.12295766],
       [  65.33255763],
       [ 141.80223878],
       [1566.16246224],
       [  12.32450391],
       [-315.34552489]])